In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'gggsdsd:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5281104%2F8784850%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240709%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240709T152624Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D59e243152c5ab0356f916d603479f77e2cf8bed7406393c9c1353cc8c8a23b98a145b650c827f1aa02ec6ae68318eab63ecd526d3a7dcf67bcc9231411a24b59be6c0be88a676ce1ec2ba7e88329c054f34ab241af214cdb1bc45fde33de2f09bf897fe052739caabdd37964a1a00a4e08cfd842a9d904ecbe0a14e6167738c847de1e454a097b9d771bb920ae2471a618c50beb7557b447c80e5197647ec82299f133f3e06e73c3663c2836f6c3b779b27ff3ba2c3f89d81d194eb59a586fb75842c693c4c092ca7154df7bdc8b128b7485ef77e85f3f4057b2d7a405eff36d4c99a0cb5f79887443e69d246ca14018cccc31c4f0517549429426029ced1ee1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gggsdsd/balanced.csv


In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/gggsdsd/balanced.csv',encoding='utf-8')
df.head()

,texts,label
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0
2,हमे इन्तजार है ।,1
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1
4,जयपुर,0


In [ ]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 13.6 MB/s eta 0:00:0000:010:01


In [ ]:
import stanza
stanza.download('hi')
nlp = stanza.Pipeline('hi')

def get_pos_tags(sentence):
    doc = nlp(sentence)
    return [word.upos for sent in doc.sentences for word in sent.words]

df['pos'] = df['texts'].apply(get_pos_tags)
print(df['pos'])

0        [ADJ, CCONJ, ADJ, NOUN, ADP, ADJ, NOUN, ADP, A...
1        [NOUN, PRON, PROPN, PROPN, ADP, ADP, NOUN, ADP...
2                                 [PRON, NOUN, AUX, PUNCT]
3        [PRON, ADP, NOUN, ADP, DET, NOUN, NOUN, VERB, ...
4                                                  [PROPN]
                               ...                        
38683    [NOUN, ADP, NOUN, PROPN, ADP, VERB, SCONJ, PRO...
38684    [PROPN, ADP, ADJ, NOUN, ADP, NOUN, ADP, NOUN, ...
38685                               [AUX, AUX, AUX, PUNCT]
38686    [PRON, PRON, PRON, VERB, AUX, SCONJ, ADV, NOUN...
38687    [PRON, VERB, SCONJ, ADJ, ADV, NOUN, ADP, ADJ, ...
Name: pos, Length: 38688, dtype: object


In [ ]:
df.to_csv('df_pos.csv', index=False)